# Globular Cluster Member Identification: NGC 6397
NGC 6397 is the second closest globular cluster to Earth (second to M4).  Similar to our exploration of M4 using Gaia, we're going to make use of a catalog of confidently identified stellar members of the cluster to train classifiers capable of making such classifications from Gaia observations.

First we'll download the necessary data.  The catalog was originally pulled from [here](http://cdsarc.u-strasbg.fr/ftp/J/A+A/616/A12/), and can be found in `data/NGC6397-1.dat`.

From the Gaia archive we'll pull all objects in a 2 deg x 1.5 deg box centered on the cluster.  The query used is below, which was used to generate `data/gaia-NGC6397-neighborhood.csv`.

```sql
SELECT TOP 500000 gaia_source.source_id,gaia_source.ra,gaia_source.dec,gaia_source.parallax,gaia_source.parallax_error,gaia_source.pm,gaia_source.pmra,gaia_source.pmra_error,gaia_source.pmdec,gaia_source.pmdec_error,gaia_source.phot_g_mean_mag,gaia_source.phot_bp_mean_mag,gaia_source.phot_rp_mean_mag,gaia_source.bp_rp,gaia_source.radial_velocity,gaia_source.radial_velocity_error
FROM gaiadr3.gaia_source 
WHERE 
CONTAINS(
	POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),
	BOX('ICRS',265.17,-53.68,2,1.5)
)=1
```

# 1. Classification

1. Load the data and cross match the confident cluster members with the larger Gaia sample.

2. Explore the data, sticking to position (e.g., ra, dec, etc.) and velocity (e.g. pm, etc.) measurements for now.  Are there particular observed quantities that seem useful for distinguishing cluster members from background stars?  Be sure to include lots of figures and discussion!

3. Build and train a model (the type of model is up to you!) for classifying stars as members or non-members of NGC 6397 based on Gaia observations.  Be sure to show your process for building and improving the model.

4. How is your final model performing?  Could it have overfit the data?  Is it clear what the model learned?

5. Does your model find any new cluster member candidates?  If so, explore their properties.  Do they seem compelling?  Be sure to connect this discussion back to your response to 1.1.

# 2. Isochrones

Recall that stellar clusters are gravitationally bound groupings of stars born at the same time.  By looking at the H-R diagram of a cluster -- specifically where the population _leaves_ the main sequence -- we can determine the age of the cluster.  To do this we must know the typical main sequence lifetime of stars, which depends on their mass, metalicity, etc. and for that we must turn to stellar modeling.  [MESA](https://docs.mesastar.org) is a stellar modeling library which incorporates the physics important for stellar evolution into 1-D models of stars.  The results useful for our purposes are stellar evolution tracks, the expected trajectory of the star through the H-R diagram as it evolves in time.

[MIST](http://waps.cfa.harvard.edu/MIST/index.html) is an effort to supply a large database of MESA stellar track simulations for the purpose of producing isochrones.  If you imagine stacking up a cluster's worth of stellar evolution tracks, slicing through them at fixed age will produce an isochrone.

I've used the [MIST isochrone interpolator](http://waps.cfa.harvard.edu/MIST/interp_isos.html) to produce an isochrone based on known properties of NGC 6397 (metalicity, reddening due to dust, etc.), and saved `BP-RP`, `Gaia_G_EDR3=phot_g_mean_mag` (with distance modulus already applied), and phase (indicating stellar evolutionary phase) to a CSV file, `data/NGC6397_iso.csv`.

1. Read in and plot the isochrone in all its messy glory.  Try encoding stellar phase information (e.g., using color) if you can, to get a better grasp on the various stellar phases we're looking at (remember, more info on what this indicates can be found [here](http://waps.cfa.harvard.edu/MIST/README_tables.pdf)).

2. Plot all of the Gaia data we pulled along with the isochrone.  In a new figure, plot the cluster members identified in `NGC6397-1.dat` with the isochrone.

3. Use the isochrone to argue whether any new cluster candidates your classifier found are viable.

4. Now include color and brightness information in your model and retrain your classifier.  Does it perform any better?  Did you expect the outcome?

# Graduate Students

1. Use a completely different technique to classify cluster members.  How do your results compare to previous attempts?

2. Do you think your models have learned anything useful for identifying members of other clusters?  Why or why not?

3. Can you think of ways we could use the isochrone explicitly to improve our model?